# json-flattener: command line examples

This walks through some examples of running json-flattener on the command line.

You can run this notebook if you check out the github repo OR you can run on mybinder:

https://mybinder.org/v2/gh/cmungall/json-flattener/main?filepath=EXAMPLE.ipynb

## Command Line help

You can always run `--help` to get the full options.

In [1]:
!jfl --help

Usage: jfl [OPTIONS] COMMAND [ARGS]...

  Main

  Args:

      verbose (int): Verbose.     quiet (bool): Quiet.

  Returns:

      None.

Options:
  -v, --verbose
  -q, --quiet TEXT
  --help            Show this message and exit.

Commands:
  flatten    Flatten a file to TSV/CSV Example: jfl flatten --input my.yaml...
  unflatten  Unflatten a file from TSV/CSV Example: jfl unflatten --input...


### flatten command

jfl has two sub-commands, `flatten` and `unflatten`. We will get to unflatten later. To get help on `flatten`:

In [2]:
!jfl flatten --help

Usage: jfl flatten [OPTIONS]

  Flatten a file to TSV/CSV

  Example:     jfl flatten --input my.yaml --output my.tsv

  Args:

      input (str): The path to the input YAML or JSON file     output (str):
      The path to the output file.     output_format (str): The format to
      which the TSV should be converted.

Options:
  -i, --input PATH             Input file, e.g. a SSSOM tsv file.  [required]
  -I, --input-format TEXT      The string denoting the input format, e.g.
                               tsv,csv,yaml,json

  -o, --output TEXT            Output file, e.g. a SSSOM tsv file.
  -t, --output-format TEXT     Desired output format, e.g. tsv,csv,yaml,json
  -L, --multivalued-keys TEXT  List of keys that are multivalued
  -F, --flatten-keys TEXT      List of keys that are to be flattened
  -s, --serializer TEXT        Serializer to use for complex keys
  -S, --serialized-keys TEXT   List of keys that are to be serialized using
                               the serializer

 

## Example Input file

This example uses a JSON input file containing a list of book series. Each book series holds one or more book objects. Both series and books and hold other objects such as creators and reviews.

Note the input could be either JSON or YAML. Here we demonstrate with a pre-prepared JSON file [examples/books1.json](examples/books1.json)

In [3]:
# Use JQ to show the JSON structure
!jq . examples/books1.json

{
  "all_book_series": [
    {
      "books": [
        {
          "price": 5.99,
          "id": "S001.1",
          "summary": "Hobbits",
          "name": "Fellowship of the Ring"
        },
        {
          "price": 5.99,
          "summary": "More hobbits",
          "id": "S001.2",
          "name": "The Two Towers"
        },
        {
          "price": 6.99,
          "id": "S001.3",
          "summary": "Yet more hobbits",
          "name": "Return of the King"
        }
      ],
      "genres": [
        "fantasy"
      ],
      "id": "S001",
      "name": "Lord of the Rings",
      "creator": {
        "from_country": "England",
        "name": "JRR Tolkein"
      }
    },
    {
      "name": "The Culture Series",
      "creator": {
        "name": "Ian M Banks",
        "from_country": "Scotland"
      },
      "id": "S002",
      "genres": [
        "scifi"
      ],
      "books": [
        {
          "price": 5.99,
          "id": "S002.1",
          "name": "Consid

## Flattening to a TSV

Now you can use the `flatten` command to denormalize this to a TSV (the default is tab-separated)

There are a variety of options for configuring the denormalization. In this example, both the the `creator` field and `books` fields are flattened, with the latter holding an arrayL

In [4]:
!jfl flatten -C creator=flat -C books=multivalued -i examples/books1.json -o examples/books1-flattened.tsv

Note that the input JSON is not in list format, instead there is a holder object which has a key `all_book_series` that points to the list we serialize. This is selected by default.

To avoid the warning add `--key all_book_series`

Next we can look at the list on the command line:

In [6]:
!cat examples/books1-flattened.tsv

## Exploring the CSV using Pandas

Here we depart from the command line to show how to load the TSV in pandas/python

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('examples/books1-flattened.tsv', sep='\t')
df

,genres,id,name,creator_from_country,creator_name,books_summary,books_name,books_price,books_id,creator_genres
0,[fantasy],S001,Lord of the Rings,England,JRR Tolkein,[Hobbits|More hobbits|Yet more hobbits],[Fellowship of the Ring|The Two Towers|Return ...,[5.99|5.99|6.99],[S001.1|S001.2|S001.3],NaN
1,[scifi],S002,The Culture Series,Scotland,Ian M Banks,NaN,[Consider Phlebas|Player of Games],[5.99|5.99],[S002.1|S002.2],NaN
2,[scifi|fantasy],S003,Book of the New Sun,USA,Gene Wolfe,NaN,[Shadow of the Torturer|Claw of the Conciliator],[|6.99],[S003.1|S003.2],[scifi|fantasy]
3,NaN,S004,Example with single book,USA,Ms Writer,NaN,[Blah],NaN,[S004.1],[romance]
4,NaN,S005,Example with no books,USA,Mr Unproductive,NaN,NaN,NaN,NaN,[romance|scifi|fantasy]


## Storing mappings

What if we want to unflatten the TSV back into complex objects? This is possible, but it's first necessary to 
save the configuration object, as this has vital information on how the fields were mapped.

We can do the same command as before, but adding a `--save-config` option:

In [9]:
!jfl flatten --save-config examples/books1-mappings.yaml -C creator=flat -C books=multivalued -i examples/books1.json \
  -o examples/books1-flattened.tsv

In [10]:
!cat examples/books1-mappings.yaml

csv_delimiter: "\t"
csv_inner_delimiter: '|'
csv_list_markers:
- '['
- ']'
key_configs:
  books:
    delete: true
    distinct_values: null
    flatten: true
    is_list: true
    mappings:
      id: books_id
      name: books_name
      price: books_price
      summary: books_summary
    melt_list_elements: false
    serializers: []
    typemap: null
  creator:
    delete: true
    distinct_values: null
    flatten: true
    is_list: false
    mappings:
      from_country: creator_from_country
      genres: creator_genres
      name: creator_name
    melt_list_elements: false
    serializers: []
    typemap: null
sep: _


## Unflattening

A CSV that is generated using jfl should always be roundtrippable, unless lossy conversion options are specifically chosen.

First we look at the help for the unflatten command

In [11]:
!jfl unflatten --help

Usage: jfl unflatten [OPTIONS]

  Unflatten a file from TSV/CSV

  Example:     jfl unflatten --input my.tsv --output my.yaml

Options:
  -i, --input PATH             Input file, e.g. a SSSOM tsv file.  [required]
  -I, --input-format TEXT      The string denoting the input format, e.g.
                               tsv,csv,yaml,json

  -o, --output TEXT            Output file, e.g. a SSSOM tsv file.
  -t, --output-format TEXT     Desired output format, e.g. tsv,csv,yaml,json
  -L, --multivalued-keys TEXT  List of keys that are multivalued
  -F, --flatten-keys TEXT      List of keys that are to be flattened
  -s, --serializer TEXT        Serializer to use for complex keys
  -S, --serialized-keys TEXT   List of keys that are to be serialized using
                               the serializer

  -C, --config-key TEXT        Key configuration. Must be of form
                               KEY={yaml,json,flat,multivalued}*

  -c, --load-config TEXT       Path to global configuration fil

In [12]:
!jfl unflatten -i examples/books1-flattened.tsv --load-config examples/books1-mappings.yaml \
  -o examples/books1-roundtripped.json

In [13]:
!jq . examples/books1-roundtripped.json

[
  {
    "genres": [
      "fantasy"
    ],
    "id": "S001",
    "name": "Lord of the Rings",
    "books": [
      {
        "id": "S001.1",
        "name": "Fellowship of the Ring",
        "price": 5.99,
        "summary": "Hobbits"
      },
      {
        "id": "S001.2",
        "name": "The Two Towers",
        "price": 5.99,
        "summary": "More hobbits"
      },
      {
        "id": "S001.3",
        "name": "Return of the King",
        "price": 6.99,
        "summary": "Yet more hobbits"
      }
    ],
    "creator": {
      "from_country": "England",
      "name": "JRR Tolkein"
    }
  },
  {
    "genres": [
      "scifi"
    ],
    "id": "S002",
    "name": "The Culture Series",
    "books": [
      {
        "id": "S002.1",
        "name": "Consider Phlebas",
        "price": 5.99
      },
      {
        "id": "S002.2",
        "name": "Player of Games",
        "price": 5.99
      }
    ],
    "creator": {
      "from_country": "Scotland",
      "name": "Ian M Bank